# Server.py 
## Federated Learning Server
Accepts connection from a set number of clients. Has access to a repository of unlabeled public data. Once all clients send predictions on public data, server aggregates results. Once resulrs are compiled the server sends out the public dataset to supliment client training with the newly assigned labels.

In [33]:
import send_receive
import socket
import threading
import torchvision.datasets as datasets
import numpy as np

In [34]:
def load_dataset():
    mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
    trainimages = mnist_trainset.data[50000:,:] #load the last 10,000 images leaving the rest for the clients private data
    return trainimages

In [35]:
NUM_CLIENTS = 10
NUM_ROUNDS = 10
NUM_CLASSES = 10

logits_dict = {}
num_responses = 0
agreggation_done = 0

public_data = load_dataset()
labels = np.empty(len(public_data))
print(public_data.shape)

torch.Size([10000, 28, 28])


In [36]:
def aggregation_rule(logits_dict):
    global labels
    #TODO: Define aggregation [logits(CLIENTS, IMAGES, CLASSES)->in, labels(IMAGES)->out]
    aggregated_logits = np.empty_like(labels)
    labels = aggregated_logits


In [37]:
def handle_client(conn, addr, public_data):
    global num_repsonses
    global logits_dict
    global labels
    print(f"[+] Connected: {addr}")

    try:
        # INITIAL ROUND
        # 1. Send initial public data
        send_receive.send_numpy(conn, public_data.numpy())

        # 2. recieve public data logits
        logits = send_receive.recv_numpy(conn)

        #TODO: Aggregate
        #Then update global var logit
        logits_dict[threading.get_ident()] = logits
        num_repsonses += 1

        if num_responses >= NUM_CLIENTS:
            aggregation_rule(logits_dict)
        
        # SUBSEQUENT ROUNDS
        for i in range(NUM_ROUNDS):
            send_receive.send_numpy(conn, labels.numpy())

            logits = send_receive.recv_numpy(conn)

            logits_dict[threading.get_ident()] = logits
            num_responses += 1

            if num_responses >= NUM_CLIENTS:
                aggregation_rule(logits_dict)

    except ConnectionResetError:
        print(f"[-] Connection reset by {addr}")
    finally:
        conn.close()
        print(f"[-] Disconnected: {addr}")

def start_server(HOST, PORT):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((HOST, PORT))
    server.listen()

    print(f"[SERVER] Listening on {HOST}:{PORT}")



    while True:
        conn, addr = server.accept()
        thread = threading.Thread(target=handle_client, args=(conn, addr, public_data), daemon=True)
        thread.start()


In [ ]:
HOST = "127.0.0.1"
PORT = 65434
start_server(HOST, PORT)

OSError: [Errno 48] Address already in use